In [29]:
import pandas as pd
from bs4 import BeautifulSoup
import threading
import concurrent.futures

In [30]:
df = pd.read_csv('./filtered.csv')
df = df.sort_values(by='Pret')

In [31]:
df.columns

Index(['Unnamed: 0', 'Pret', 'Oferit de', 'Are VIN (Serie sasiu)', 'Marca',
       'Model', 'Versiune', 'Anul fabricației', 'Km', 'Combustibil', 'Putere',
       'Capacitate cilindrica', 'Transmisie', 'Norma de poluare',
       'Cutie de viteze', 'Tip Caroserie', 'Numar de portiere', 'Culoare',
       'Numar locuri', 'Optiuni culoare', 'Se emite factura',
       'Eligibil pentru finantare', 'Garantie de la producator pana la',
       'sau in limita a', 'Tara de origine', 'Inmatriculat',
       'Primul proprietar (de nou)', 'Fara accident in istoric', 'Stare'],
      dtype='object')

In [32]:
df['Stare']

4475    Second hand
3772    Second hand
4161    Second hand
4389    Second hand
2370    Second hand
           ...     
550     Second hand
619     Second hand
4133    Second hand
2738    Second hand
3518    Second hand
Name: Stare, Length: 9293, dtype: object

In [33]:
df.head(10000)

,Unnamed: 0,Pret,Oferit de,Are VIN (Serie sasiu),Marca,Model,Versiune,Anul fabricației,Km,Combustibil,...,Optiuni culoare,Se emite factura,Eligibil pentru finantare,Garantie de la producator pana la,sau in limita a,Tara de origine,Inmatriculat,Primul proprietar (de nou),Fara accident in istoric,Stare
4475,15652,1 100 EUR,Privat,Da,Seat,Cordoba,1.4 Signo,2004,164 000 km,Benzina,...,Metalizata,Nu,Nu,Nu,Nu,Romania,Da,Nu,Da,Second hand
3772,13033,1 500 EUR,Privat,Da,Ford,Mondeo,Turnier 2.0 TDCi Ghia,2006,313 958 km,Diesel,...,Perlat,Nu,Nu,Nu,Nu,Franta,Da,Nu,Da,Second hand
4161,14492,1 900 EUR,Privat,Nu,Opel,Astra,1.6i Cosmo,2005,198 376 km,Benzina,...,Metalizata,Nu,Nu,Nu,Nu,Germania,Nu,Nu,Nu,Second hand
4389,15323,1 950 EUR,Privat,Da,Ford,Focus,Turnier 1.6 TDCi DPF Style,2005,213 000 km,Diesel,...,Metalizata,Nu,Nu,Nu,Nu,Germania,Da,Nu,Da,Second hand
2370,7791,1 950 EUR,Firma,Nu,Renault,Scenic,1.6 16V Exception,2005,260 000 km,Benzina,...,Metalizata,Da,Da,Nu,Nu,Germania,Da,Nu,Da,Second hand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,1512,99 949 EUR,Firma,Da,BMW,X6,xDrive30d AT MHEV,2023,7 750 km,Diesel,...,Metalizata,Da,Da,12 Septembrie 2028,200 000 km,Romania,Da,Da,Da,Second hand
619,1659,99 960 EUR,Firma,Nu,Mercedes-Benz,S,350 d 4Matic 9G-TRONIC,2021,39 820 km,Diesel,...,Metalizata,Da,Da,Nu,Nu,Germania,Nu,Da,Da,Second hand
4133,14362,99 960 EUR,Firma,Nu,Land Rover,Range Rover Sport,5.0 V8 Supercharged SVR Dynamic,2021,17 600 km,Benzina,...,Mat,Da,Da,Nu,Nu,Germania,Da,Da,Da,Second hand
2738,9073,99 975 EUR,Firma,Da,Mercedes-Benz,GLE Coupe,400 d 4Matic 9G-TRONIC AMG Line,2022,9 584 km,Diesel,...,Metalizata,Da,Da,Nu,200 000 km,Germania,Nu,Da,Da,Second hand


In [34]:
df = df.drop(columns = ['Unnamed: 0'])
#df.head()

In [35]:
def getDateScore(str):
    #print(str)
    day,month,year = str.split()

    return int(year)*12+int(month)

In [36]:
currentDate = '11 1 2024'
monthDict = {'Ianuarie':1,'Februarie':2,'Martie':3,'Aprilie':4,'Mai':5,'Iunie':6,'Iulie':7,'August':8,'Septembrie':9,'Octombrie':10,'Noiembrie':11,'Decembrie':12}

In [37]:
for x,y in monthDict.items():
    df['Garantie de la producator pana la'] = df['Garantie de la producator pana la'].apply(lambda z: z.replace(x,str(y)))


In [38]:
#print(set(df['Garantie de la producator pana la']))

In [39]:
df['Pret'] = df['Pret'].apply(lambda x : int("".join(x.split()[:-1])))
df['Km'] = df['Km'].apply(lambda x : int("".join(x.split()[:-1])))
df['Putere'] = df['Putere'].apply(lambda x : int("".join(x.split()[:-1])))
df['Capacitate cilindrica'] = df['Capacitate cilindrica'].apply(lambda x : int("".join(x.split()[:-1])))
df['sau in limita a'] = df['sau in limita a'].apply(lambda x: int("".join(x.split()[:-1])) if str(x)[-1]=='m' else 0)
df['sau in limita a'] = df['sau in limita a'] - df['Km']
df['sau in limita a'] = df['sau in limita a'].apply(lambda x: x if x>=0 else 0)
df['Anul fabricației'] = df['Anul fabricației'].apply(lambda x: 2024 - int(x))
df['Garantie de la producator pana la'] = df['Garantie de la producator pana la'].apply(lambda x: 0 if str(x)=='Nu' else max(0,getDateScore(x) - getDateScore(currentDate)))
df = df.rename(columns={'Anul fabricației': 'Vechime'})


df = df.replace("Da",1)
df = df.replace("Nu",0)
df = df.sort_values('sau in limita a',ascending =False)
df.head(500)

,Pret,Oferit de,Are VIN (Serie sasiu),Marca,Model,Versiune,Vechime,Km,Combustibil,Putere,...,Optiuni culoare,Se emite factura,Eligibil pentru finantare,Garantie de la producator pana la,sau in limita a,Tara de origine,Inmatriculat,Primul proprietar (de nou),Fara accident in istoric,Stare
4134,71899,Firma,1,BMW,X5,xDrive45e,2,11300,Hibrid,286,...,Perlat,1,1,15,988699,Germania,0,0,1,Second hand
1183,19989,Firma,0,Opel,Astra,1.5 Start/Stop Elegance,2,11700,Diesel,130,...,Metalizata,1,1,40,988299,Romania,1,1,1,Second hand
1286,59899,Firma,1,BMW,M4,M440i xDrive AT MHEV,1,25400,Benzina,374,...,Metalizata,1,1,26,974599,Germania,0,0,1,Second hand
3006,30900,Firma,1,BMW,X1,xDrive25e Advantage,3,26100,Hibrid,125,...,Metalizata,1,1,5,973899,Germania,0,0,1,Second hand
1378,56899,Firma,1,BMW,Seria 5,545e xDrive Aut.,1,27500,Hibrid,286,...,Metalizata,1,1,24,972499,Germania,0,0,1,Second hand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667,35498,Firma,1,Mercedes-Benz,A,250 e Aut.,4,30095,Hibrid,160,...,Mat,1,1,0,169905,Germania,1,0,1,Second hand
8645,35498,Firma,1,Mercedes-Benz,A,250 e Aut.,4,30095,Hibrid,160,...,Mat,1,1,0,169905,Germania,1,0,1,Second hand
7451,35498,Firma,1,Mercedes-Benz,A,250 e Aut.,4,30095,Hibrid,160,...,Mat,1,1,0,169905,Germania,1,0,1,Second hand
7950,35498,Firma,1,Mercedes-Benz,A,250 e Aut.,4,30095,Hibrid,160,...,Mat,1,1,0,169905,Germania,1,0,1,Second hand


In [40]:
#df = df.sort_values('sau in limita a',ascending =False)
#df.head()

In [41]:
df = df.drop(columns = ['Versiune'])
df.head()


,Pret,Oferit de,Are VIN (Serie sasiu),Marca,Model,Vechime,Km,Combustibil,Putere,Capacitate cilindrica,...,Optiuni culoare,Se emite factura,Eligibil pentru finantare,Garantie de la producator pana la,sau in limita a,Tara de origine,Inmatriculat,Primul proprietar (de nou),Fara accident in istoric,Stare
4134,71899,Firma,1,BMW,X5,2,11300,Hibrid,286,2998,...,Perlat,1,1,15,988699,Germania,0,0,1,Second hand
1183,19989,Firma,0,Opel,Astra,2,11700,Diesel,130,1498,...,Metalizata,1,1,40,988299,Romania,1,1,1,Second hand
1286,59899,Firma,1,BMW,M4,1,25400,Benzina,374,2998,...,Metalizata,1,1,26,974599,Germania,0,0,1,Second hand
3006,30900,Firma,1,BMW,X1,3,26100,Hibrid,125,1499,...,Metalizata,1,1,5,973899,Germania,0,0,1,Second hand
1378,56899,Firma,1,BMW,Seria 5,1,27500,Hibrid,286,2998,...,Metalizata,1,1,24,972499,Germania,0,0,1,Second hand


In [43]:
df= pd.get_dummies(df, columns=[ 'Oferit de', 'Are VIN (Serie sasiu)', 'Marca',
         'Combustibil',
        'Transmisie', 'Norma de poluare',
       'Cutie de viteze', 'Tip Caroserie',  'Culoare',
        'Optiuni culoare',
        'Tara de origine', 'Stare','Model'])

In [50]:
# target_column = df['Pret']

# # Dictionary to hold correlations
# correlations = {}

# # Calculating correlation of each column in df1 with the target column in df2
# for column in df.columns:
#     correlation = df[column].corr(target_column)
#     correlations[column] = correlation

# # Converting the dictionary to a DataFrame for better visualization
# correlation_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation'])

# print(list(correlation_df['Correlation']))


In [16]:
df.describe()

,Pret,Vechime,Km,Putere,Capacitate cilindrica,Numar de portiere,Numar locuri,Se emite factura,Eligibil pentru finantare,Garantie de la producator pana la,sau in limita a,Inmatriculat,Primul proprietar (de nou),Fara accident in istoric
count,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000,9293.000000
mean,29038.944582,5.640159,103587.635855,187.462176,1950.979877,4.677069,4.901216,0.646401,0.639621,6.214570,22564.852147,0.556440,0.662004,0.869256
std,23615.166455,3.448388,77249.949486,91.098680,658.855593,0.607731,0.737306,0.478112,0.480136,21.575474,59670.124371,0.496831,0.473053,0.337138
min,1100.000000,0.000000,1.000000,54.000000,799.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13990.000000,4.000000,30095.000000,115.000000,1461.000000,4.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,23205.000000,5.000000,100000.000000,160.000000,1968.000000,5.000000,5.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
75%,35990.000000,6.000000,152000.000000,235.000000,2199.000000,5.000000,5.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,299999.000000,24.000000,510000.000000,650.000000,6592.000000,5.000000,9.000000,1.000000,1.000000,120.000000,988699.000000,1.000000,1.000000,1.000000


                       Correlation
Pret                      1.000000
Vechime                  -0.424440
Km                       -0.455837
Putere                    0.765066
Capacitate cilindrica     0.643372
...                            ...
Model_i10                -0.009528
Model_i20                -0.013271
Model_i40                -0.015301
Model_ix35               -0.030719
Model_up!                -0.013658

[491 rows x 1 columns]


In [17]:
df.to_csv('./preprocessed.csv')

In [18]:
train_labels = df[['Pret']]
train_samples = df.drop('Pret', axis=1)

In [19]:
from sklearn.model_selection import train_test_split
import numpy as np



In [20]:
#!pip install scikit-learn

In [25]:
X_train, X_val, y_train, y_val = train_test_split(train_samples, train_labels, test_size=0.05, random_state=42)

In [26]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train,y_train)
X_val = scaler.transform(X_val)

C:\Users\Ionut Anghelina\.conda\envs\mlenvfinal\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error: 14180.784458780108
Root Mean Squared Error: 20630.930646854587
